# import library

In [36]:
!pip install dgl

In [37]:
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl
from dgl.data import CoraGraphDataset
from sklearn.metrics import f1_score
import dgl.function as fn

In [38]:
G = CoraGraphDataset()
num_classes = G.num_classes
G = G[0]

features = G.ndata['feat']
input_feature_dim = features.shape[1]
labels = G.ndata['label']
train_mask = G.ndata['train_mask']
test_mask = G.ndata['test_mask']

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [39]:
# 하이퍼파라미터 정의
lr = 1e-2
num_epochs = 50
num_hidden_dim = 128
num_layers = 2
weight_decay = 5e-4

# GraphSAGE
mean as AGG function
$$
\mathbf{h}_v^k = \sigma(W_k\cdot \left[ AGG(\{\mathbf{h}_u^{k-1}, \forall k\in N(v)\}), \mathbf{h}_v^{k-1} \right]) \\
AGG = \sum_{u\in N(v)}\frac{\mathbf{h}_u^{k-1}}{|N(v)|}
$$

In [40]:
class SAGEConv(nn.Module):
    def __init__(self, in_feats, out_feats, activation):
        super(SAGEConv, self).__init__()
        self._in_feats = in_feats
        self._out_feats = out_feats
        self.activation = activation
        self.W = nn.Linear(in_feats + in_feats, out_feats, bias=True)
    
    def forward(self, graph, feature):
        graph.ndata['h'] = feature
        graph.update_all(fn.copy_src('h', 'm'), fn.sum('m', 'neigh'))
        degs = graph.in_degrees().to(feature)
        hk_neigh = graph.ndata['neigh'] / degs.unsqueeze(-1)
        hk = self.W(torch.cat((graph.ndata['h'], hk_neigh), dim = -1))
        if self.activation != None:
            hk = self.activation(hk)
        return hk


In [41]:
class GraphSAGE(nn.Module):
    def __init__(self, graph, in_feat_dim, num_hidden_dim, num_classes, num_layers, activation_fun):
        super(GraphSAGE, self).__init__()
        self.layers = nn.ModuleList()
        self.graph = graph
        self.layers.append(SAGEConv(in_feat_dim, num_hidden_dim, activation_fun))
        for i in range(num_layers):
            self.layers.append(SAGEConv(num_hidden_dim, num_hidden_dim, activation_fun))
        self.layers.append(SAGEConv(num_hidden_dim, num_classes, activation = None))
    
    def forward(self, features):
        x = features
        for layer in self.layers:
            x = layer(self.graph, x)
        return x


In [42]:
model = GraphSAGE(G, input_feature_dim, num_hidden_dim, num_classes, num_layers, F.relu)

In [43]:
# 모델 학습 결과를 평가할 함수
def evaluate_train(model, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

def evaluate_test(model, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        macro_f1 = f1_score(labels, indices, average = 'macro')
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels), macro_f1

In [50]:
def train(model, loss_fun, features, labels, train_mask, optim, num_epochs):
    running_time = []
    for epoch in range(num_epochs):
        model.train()
        start_t = time.time()
        logits = model(features)
        loss = loss_fun(logits[train_mask], labels[train_mask])
        optim.zero_grad()
        loss.backward()
        optim.step()
        running_time.append(time.time()-start_t)
        acc = evaluate_train(model, features, labels, train_mask)

        print(f"Epoch: {epoch}\tTime(s): {running_time[epoch]}\tLoss: {loss.item():.4f}\tAcc: {acc:.6f}")

def test(model, features, labels, test_mask):
    acc, macro_f1 = evaluate_test(model, features, labels, test_mask)
    print(f"Test Accuracy:\t{acc: .6f}")
    print(f"Test macro-f1:\t{macro_f1: .4f}")

# train & test

In [45]:
model = GraphSAGE(G, input_feature_dim, num_hidden_dim, num_classes, num_layers, F.relu)
print(model)

GraphSAGE(
  (layers): ModuleList(
    (0): SAGEConv(
      (W): Linear(in_features=2866, out_features=128, bias=True)
    )
    (1): SAGEConv(
      (W): Linear(in_features=256, out_features=128, bias=True)
    )
    (2): SAGEConv(
      (W): Linear(in_features=256, out_features=128, bias=True)
    )
    (3): SAGEConv(
      (W): Linear(in_features=256, out_features=7, bias=True)
    )
  )
)


In [46]:
loss_fun = torch.nn.CrossEntropyLoss()

optim = torch.optim.Adam(model.parameters(), lr = lr, weight_decay = weight_decay)

In [47]:
train(model, loss_fun, features, labels, train_mask, optim, num_epochs)

Epoch: 0	Time(s): 0.2809939384460449	Loss: 1.9466	Acc: 0.142857
Epoch: 1	Time(s): 0.1759033203125	Loss: 1.9460	Acc: 0.142857
Epoch: 2	Time(s): 0.17191576957702637	Loss: 1.9445	Acc: 0.142857
Epoch: 3	Time(s): 0.17142105102539062	Loss: 1.9414	Acc: 0.500000
Epoch: 4	Time(s): 0.17311668395996094	Loss: 1.9292	Acc: 0.421429
Epoch: 5	Time(s): 0.1741025447845459	Loss: 1.8883	Acc: 0.564286
Epoch: 6	Time(s): 0.17392563819885254	Loss: 1.7925	Acc: 0.435714
Epoch: 7	Time(s): 0.17014765739440918	Loss: 1.6489	Acc: 0.700000
Epoch: 8	Time(s): 0.17030668258666992	Loss: 1.4474	Acc: 0.707143
Epoch: 9	Time(s): 0.17139840126037598	Loss: 1.1951	Acc: 0.742857
Epoch: 10	Time(s): 0.1704564094543457	Loss: 0.9364	Acc: 0.807143
Epoch: 11	Time(s): 0.16858124732971191	Loss: 0.7191	Acc: 0.892857
Epoch: 12	Time(s): 0.1721210479736328	Loss: 0.4661	Acc: 0.900000
Epoch: 13	Time(s): 0.16767358779907227	Loss: 0.3151	Acc: 0.892857
Epoch: 14	Time(s): 0.17084908485412598	Loss: 0.3225	Acc: 0.942857
Epoch: 15	Time(s): 0.1710782

In [51]:
test(model, features, labels, test_mask)

Test Accuracy:	 0.693000
Test macro-f1:	 0.6863
